# Data Processing

## import packages

In [1]:
import numpy as np
import pandas as pd
from arbitragerepair import constraints, repair

In [2]:
raw_data = pd.read_csv('fqjo3s8eacwzxkcw.csv')

In [3]:
# Choose Amazon as the underlying stock
df = raw_data.loc[raw_data.ticker=='AMZN', 
                  ['date','exdate','cp_flag','strike_price','best_bid','best_offer','volume','impl_volatility']].copy()
df['date'] = pd.to_datetime(df['date'])
df['exdate'] = pd.to_datetime(df['exdate'])
df['strike_price'] = df['strike_price']/1000 # scaling strike
df['option_price'] = df[['best_bid','best_offer']].mean(axis=1)
df = df.drop(columns = ['best_bid','best_offer'])
df.head()

,date,exdate,cp_flag,strike_price,volume,impl_volatility,option_price
0,2018-01-02,2018-01-05,C,1000.0,0,0.962943,189.975
1,2018-01-02,2018-01-05,C,1002.5,0,0.951017,187.475
2,2018-01-02,2018-01-05,C,1005.0,0,0.939104,184.975
3,2018-01-02,2018-01-05,C,1007.5,0,0.927205,182.475
4,2018-01-02,2018-01-05,C,1010.0,0,0.915319,179.975


## Revise stock prices

### Upload stock data


In [7]:
import datetime as dt
import yfinance as yf
amzn = yf.Ticker("AMZN")
#amzn.info
#amzn.institutional_holders

start_date = dt.datetime(2018, 1, 1)

tickers_list = ['AMZN']

df_amzn = yf.download(tickers_list, start=start_date, progress=True)
print(df_amzn.head())
fn = './amzn_stock_daily.csv'
df_amzn.to_csv(fn, sep=',', encoding='utf-8')
print('saved to %s'%(fn))

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-01-02  58.599998  59.500000  58.525501  59.450500  59.450500  53890000
2018-01-03  59.415001  60.274502  59.415001  60.209999  60.209999  62176000
2018-01-04  60.250000  60.793499  60.233002  60.479500  60.479500  60442000
2018-01-05  60.875500  61.457001  60.500000  61.457001  61.457001  70894000
2018-01-08  61.799999  62.653999  61.601501  62.343498  62.343498  85590000
saved to ./amzn_stock_daily.csv


In [9]:
df_amzn.reset_index(inplace=True)
df_amzn['date'] = pd.to_datetime(df_amzn['Date'], format="%d/%m/%y")
stock_AMZN = df_amzn[['date','Adj Close']].copy()
stock_AMZN.rename(columns = {'Adj Close' : 'stock_price'}, inplace = True)

split_data = {
    'date': ['2022-06-06', '1999-09-02', '1999-01-05', '1998-06-02'],
    'split': ['20:1', '2:1', '3:1', '2:1'],
    'multiple': [20, 2, 3, 2]
}
df_splits = pd.DataFrame(split_data)
df_splits['date'] = pd.to_datetime(df_splits['date'])
df_splits = df_splits.sort_values(by='date', ascending=False)

# Define a function to adjust stock prices based on splits.
def adjust_prices_for_splits(stock_prices_df, splits_df):
    adjusted_prices = stock_prices_df.copy()
    adjusted_prices['adjusted_price'] = adjusted_prices['stock_price']
    
    # Apply splits in reverse chronological order
    for _, split in splits_df.iterrows():
        split_date = split['date']
        split_multiple = split['multiple']
        mask = adjusted_prices['date'] < split_date
        adjusted_prices.loc[mask, 'adjusted_price'] *= split_multiple
        
    return adjusted_prices

# Adjust the AMZN stock prices for splits
stock_AMZN = adjust_prices_for_splits(stock_AMZN, df_splits)
stock_AMZN

,date,stock_price,adjusted_price
0,2018-01-02,59.450500,1189.010010
1,2018-01-03,60.209999,1204.199982
2,2018-01-04,60.479500,1209.589996
3,2018-01-05,61.457001,1229.140015
4,2018-01-08,62.343498,1246.869965
...,...,...,...
1630,2024-06-26,193.610001,193.610001
1631,2024-06-27,197.850006,197.850006
1632,2024-06-28,193.250000,193.250000
1633,2024-07-01,197.199997,197.199997


### Process different pairs of data, with 3 months difference for T2 and T1

In [46]:
# Arbitrage repair function
# pip install pandas_market_calendars
import pandas_market_calendars as mcal
def arbitrageRepair(columnT, columnK, columnC, columnF):
    # normalise strikes and call prices
    normaliser = constraints.Normalise()
    T = np.array(columnT)
    K = np.array(columnK)
    C = np.array(columnC)
    F = np.array(columnF)
    normaliser.fit(T, K, C, F)
    T1, K1, C1 = normaliser.transform(T, K, C)
    
    # construct arbitrage constraints and detect violation
    mat_A, vec_b, _, _ = constraints.detect(T1, K1, C1, verbose=False)
    
    # repair arbitrage - l1-norm objective
    epsilon = repair.l1(mat_A, vec_b, C1)
    
    # de-normalise
    K0, C0 = normaliser.inverse_transform(K1, C1 + epsilon)
    
    return K0, C0


def dataProcess2Di(df,t1,t2,ticker):

    # call options data
    df_t1_C = df.loc[(df.exdate==t1)&(df.cp_flag=='C'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]
    df_t2_C = df.loc[(df.exdate==t2)&(df.cp_flag=='C'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]

    # put options data
    df_t1_P = df.loc[(df.exdate==t1)&(df.cp_flag=='P'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]
    df_t2_P = df.loc[(df.exdate==t2)&(df.cp_flag=='P'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]

    t0List = list(df_t1_C.date.unique())

    result = pd.DataFrame()

    for t0 in t0List:
        ## T1:
        # t1: 20 largest trading volume
        df_t0t1 = df_t1_C[df_t1_C.date==t0].sort_values('volume',ascending=False).iloc[0:20].drop(columns='volume').reset_index(drop=True)

        # merge with put options data and stock price data
        df_t0t1 = df_t0t1.merge(df_t1_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t1 = pd.merge(df_t0t1, stock_AMZN, on = 'date')

        # calculate the forward price:
        df_t0t1['forward_price'] = df_t0t1['adjusted_price']/((df_t0t1['adjusted_price']-(df_t0t1['option_price_C']-df_t0t1['option_price_P']))/df_t0t1['strike_price'])
        #df_t0t1['forward_price'] = df_t0t1['adjusted_price']

        # calculate the expiry:
        # Amazon (AMZN) is listed on the NASDAQ exchange
        amzn_t = mcal.get_calendar('NASDAQ')
        trading_days_1 = amzn_t.valid_days(start_date=t0, end_date=t1)
        df_t0t1['expiry'] = len(trading_days_1) / 252
        #df_t0t1['expiry'] = (df_t0t1['exdate']-df_t0t1['date']).apply(lambda x: x.days/365)

        # arbitrage repair:
        K0_t1, C0_t1 = arbitrageRepair(df_t0t1['expiry'], df_t0t1['strike_price'], df_t0t1['option_price_C'], df_t0t1['forward_price'])

        # temporary result at t0:
        result_t1 = df_t0t1[['date','exdate']].copy()
        result_t1['strike_price'] = K0_t1
        result_t1['call_option_price'] = C0_t1


        ## T2:
        # t2: 20 largest trading volume
        df_t0t2 = df_t2_C[df_t2_C.date==t0].sort_values('volume',ascending=False).iloc[0:20].drop(columns='volume').reset_index(drop=True)

        # merge with put options data and stock price data
        df_t0t2 = pd.merge(df_t0t2, df_t2_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t2 = pd.merge(df_t0t2, stock_AMZN, on = 'date')

        # calculate the forward price:
        df_t0t2['forward_price'] = df_t0t2['adjusted_price']/((df_t0t2['adjusted_price']-(df_t0t2['option_price_C']-df_t0t2['option_price_P']))/df_t0t2['strike_price'])
        # df_t0t2['forward_price'] = df_t0t2['adjusted_price']

        # calculate the expiry:
        trading_days_2 = amzn_t.valid_days(start_date=t0, end_date=t2)
        df_t0t2['expiry'] = len(trading_days_2) / 252
        # df_t0t2['expiry'] = (df_t0t2['exdate']-df_t0t2['date']).apply(lambda x: x.days/365)

        # arbitrage repair:
        K0_t2, C0_t2 = arbitrageRepair(df_t0t2['expiry'], df_t0t2['strike_price'], df_t0t2['option_price_C'], df_t0t2['forward_price'])

        # temporary result at t0:
        result_t2 = df_t0t2[['date','exdate']].copy()
        result_t2['strike_price'] = K0_t2
        result_t2['call_option_price'] = C0_t2


        ## Concat horizontally  
        result_tmp = pd.concat([result_t1, result_t2.drop(columns='date')], axis=1)

        ## Concat vertically
        result = pd.concat([result,result_tmp], axis=0)

    result.columns = ['date', 'T1', 'K1', 'C1',  'T2', 'K2', 'C2']
    result = result.reset_index(drop=True)
    result = pd.merge(result, stock_AMZN, on = 'date')
    result.columns = ['t0', 'T1', 'K1', 'C1',  'T2', 'K2', 'C2',  'S0', 'Adj_S0']
    file_name = 'data/{}_{}_{}.csv'.format(ticker,pd.to_datetime(t1).strftime('%Y%m%d'),pd.to_datetime(t2).strftime('%Y%m%d'))
    result.to_csv(file_name, header=True)

In [47]:
## Select T1 and T2 which has closest difference of 3 months, arbitrage repair and save to path 'data/'
T1_List = df.loc[df.date=='2018-01-02','exdate'].unique()
T1_T2_List = pd.DataFrame()
for t1 in T1_List:
    t2 = T1_List[T1_List.year*12+T1_List.month == t1.year*12+t1.month+3]
    if len(t2) > 1:
        t2 = t2[t2.day == min(np.abs(t2.day-t1.day))]
    if len(t2) > 0:
        T1_T2_List = pd.concat([T1_T2_List,pd.DataFrame([t1,t2[0]]).T])
for t2 in T1_T2_List[1]:
    if len(T1_T2_List[T1_T2_List[1]==t2]) >1:
        df_t2 = T1_T2_List[T1_T2_List[1]==t2]
        df_t2['diff'] = np.abs(df_t2[0].dt.day - t2.day)
        df_t2 = df_t2.sort_values(by='diff').iloc[:1,:2]
        T1_T2_List = pd.concat([T1_T2_List[T1_T2_List[1]!=t2],df_t2])

for i in range(len(T1_T2_List)):
    t1 = T1_T2_List.iloc[i,0]
    t2 = T1_T2_List.iloc[i,1]
    dataProcess2Di(df,t1,t2,ticker='AMZN')

C:\Users\yanhui\AppData\Local\Temp\ipykernel_18172\3176926426.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t2['diff'] = np.abs(df_t2[0].dt.day - t2.day)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constr

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

...

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-

C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:561: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:621: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-Option-Pricing\Empirical-Option-Pricing\arbitragerepair\constraints.py:515: RuntimeWarning: invalid value encountered in cast
  raw_index = df['raw_index'].values.astype(int)
C:\Users\yanhui\Desktop\ipython\optionPricing\git-Empirical-